- references
    - https://github.com/geekplux/cv_resume

In [42]:
import inspect
import torch
from torch import nn
from peft import inject_adapter_in_model, LoraConfig

In [19]:
class DummyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = torch.nn.Embedding(100, 1000)
        self.linear = torch.nn.Linear(1000, 1000)
        self.lm_head = torch.nn.Linear(1000, 100)

    def forward(self, input_ids):
        x = self.embedding(input_ids)
        x = self.linear(x)
        x = self.lm_head(x)
        return x

In [24]:
lora_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    target_modules=["linear"],
)

In [4]:
lora_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=64, target_modules={'linear'}, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

## dummy model

In [36]:
model = DummyModel()
model

DummyModel(
  (embedding): Embedding(100, 1000)
  (linear): Linear(in_features=1000, out_features=1000, bias=True)
  (lm_head): Linear(in_features=1000, out_features=100, bias=True)
)

In [39]:
type(model.linear)

torch.nn.modules.linear.Linear

In [22]:
dummy_inputs = torch.LongTensor([[0, 1, 2, 3, 4, 5, 6, 7]])
model(dummy_inputs).shape

torch.Size([1, 8, 100])

## lora model

In [28]:
model = inject_adapter_in_model(lora_config, model)
model

DummyModel(
  (embedding): Embedding(100, 1000)
  (linear): lora.Linear(
    (base_layer): Linear(in_features=1000, out_features=1000, bias=True)
    (lora_dropout): ModuleDict(
      (default): Dropout(p=0.1, inplace=False)
    )
    (lora_A): ModuleDict(
      (default): Linear(in_features=1000, out_features=64, bias=False)
    )
    (lora_B): ModuleDict(
      (default): Linear(in_features=64, out_features=1000, bias=False)
    )
    (lora_embedding_A): ParameterDict()
    (lora_embedding_B): ParameterDict()
  )
  (lm_head): Linear(in_features=1000, out_features=100, bias=True)
)

In [40]:
type(model.embedding), type(model.linear)

(torch.nn.modules.sparse.Embedding, torch.nn.modules.linear.Linear)

In [41]:
isinstance(model.linear, nn.Module)

True

In [44]:
inspect.getmro(type(model.linear))

(torch.nn.modules.linear.Linear, torch.nn.modules.module.Module, object)